In [40]:
!pip install folium
!pip install pandas

In [41]:
import folium
import pandas as pd

In [42]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [43]:
# Download and read the `spacex_launch_geo.csv`
# Download and read the `spacex_launch_geo.csv`
import pandas as pd
import requests
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

# Use requests instead of js.fetch
response = requests.get(URL)
spacex_csv_file = io.BytesIO(response.content)
spacex_df = pd.read_csv(spacex_csv_file)

# Now you can work with spacex_df

In [44]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [45]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [46]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [47]:
# Initialize the map centered on NASA coordinate
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle and a Marker
for index, row in launch_sites_df.iterrows():
    # Add a Circle
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color='#000000',
        fill=True
    ).add_child(
        folium.Popup(row['Launch Site'])
    ).add_to(site_map)

    # Add a Marker with text label
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{row["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)



In [48]:
# Task 2: Mark the success/failed launches for each site on the map
for index, row in spacex_df.iterrows():
    # Choose color based on success (1) or failure (0)
    if row['class'] == 1:
        color = 'green'
        outcome = 'Success'
    else:
        color = 'red'
        outcome = 'Failure'
    
    # Add circle for each launch
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=500,  # smaller radius than site circle
        color=color,
        fill=True
    ).add_child(
        folium.Popup(f"Site: {row['Launch Site']}<br>Outcome: {outcome}")
    ).add_to(site_map)

In [49]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [50]:
marker_cluster = MarkerCluster()

In [51]:
# Success = green, Failure = red
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=folium.Popup(f"Site: {record['Launch Site']}<br>"
                           f"Outcome: {'Success' if record['class']==1 else 'Failure'}")
    )
    marker_cluster.add_child(marker)

# Display the map
site_map

In [52]:
from folium.plugins import MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Add MousePosition to the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)

In [53]:
# Haversine distance function
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # km radius of Earth
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [54]:
# Example: compute distance from launch site to closest coastline
launch_site_lat = 28.56342  # replace with actual launch site lat
launch_site_lon = -80.57682  # replace with actual launch site lon

coastline_lat = 28.56342  # pick from MousePosition
coastline_lon = -80.56763  # pick from MousePosition

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon,
                                        coastline_lat, coastline_lon)

In [55]:
# Add a marker for the coastline point with distance info
distance_marker = folium.Marker(
    location=[coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12px; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>'
    )
)
site_map.add_child(distance_marker)



# Display updated map
site_map

In [56]:
# Draw a line between launch site and coastline
line = folium.PolyLine(locations=[[launch_site_lat, launch_site_lon],
                                  [coastline_lat, coastline_lon]],
                       color='blue', weight=2, opacity=0.7)
site_map.add_child(line)